In [41]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import List, Optional
load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

def class_tester(cls, human_prompt, input_dict:dict):
    structured_llm = llm.with_structured_output(cls)


    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", cls.get_system_prompt()),
            ("human", human_prompt)
        ]
    )
    generator = prompt | structured_llm
    result = generator.invoke(input_dict)
    return result

In [1]:
import requests

def get_keywords(query, api_key):
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "gl": "us",
        "hl": "en"
    }
    response = requests.get("https://serpapi.com/search", params=params)
    data = response.json()

    # Extract related searches as keywords
    keywords = [related_search['query'] for related_search in data.get('related_searches', [])]
    return keywords

api_key = "acf7bee7694817980e766dfeabf80a5ca85e46441b04a81a4ad5e771fcc1f3f6"
query = "SEO efficient article"
# keywords = get_keywords(query, api_key)
keywords = ['Seo efficient article example', 'SEO articles examples', 'Best SEO articles', 'SEO article writing', 'SEO-optimized article meaning', 'What is seo', 'SEO article writer AI', 'SEO article Checker']
print(keywords)


['Seo efficient article example', 'SEO articles examples', 'Best SEO articles', 'SEO article writing', 'SEO-optimized article meaning', 'What is seo', 'SEO article writer AI', 'SEO article Checker']


In [15]:
class LongtailKeyword(BaseModel):
    """A longtail keyword is a keyword that is not in the top 10 search results."""

    keyword: str = Field(description="The longtail keyword")


class LongtailKeywordsGenerator(BaseModel):
    """Generate longtail keywords starting from a list of keywords."""

    keywords: list[LongtailKeyword] = Field(
        description="A list of generated longtail keywords."
    )

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are a longtail keyword generator. You will generate longtail keywords starting from a list of keywords."""


result = class_tester(LongtailKeywordsGenerator, "Keywords list:\n\n {kw}", {"kw": "['Seo efficient article example', 'SEO articles examples', 'Best SEO articles']"})
result


LongtailKeywordsGenerator(keywords=[LongtailKeyword(keyword='Seo efficient article example'), LongtailKeyword(keyword='SEO articles examples'), LongtailKeyword(keyword='Best SEO articles')])

In [ ]:
class KeywordChooser(BaseModel):
    """Choose a keyword from a list of keywords."""
    pass

In [20]:
article_title = "SEO article writing for e-commerce"

In [50]:
class H3Title(BaseModel):
    """A component responsible for generating the title of an H3Title"""
    
    title:str = Field(description="The title of this H2Title")
    content:str = ""

In [54]:
class H2Title(BaseModel):
    """A component responsible for generating the title of an H2Title"""
    
    title:str = Field(description="The title of this H2Title")
    content:str = Field(description="Leave empty. Will be filled later.")
    h3_titles:Optional[List[H3Title]] = Field(description="The list of H3Titles that will be generated. They are related to the H2 title.")

In [55]:
class OutlineGenerator(BaseModel):
    """Generate an outline to write a SEO efficient article from a longtail keyword."""

    h2_titles:List[H2Title] = Field(description="The generated H2Titles")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are an outline generator. You will generate an outline to write a SEO efficient article from a longtail keyword. H1 is the title of the whole article, you will include both H2Titles and H3Titles. Always include a Conclusion section at the end."""
    
    
outline = class_tester(OutlineGenerator, "Article title:\n\n {title}", {"title": article_title})
outline

OutlineGenerator(h2_titles=[H2Title(title='Introduction', content='', h3_titles=None), H2Title(title='Importance of SEO in E-commerce', content='', h3_titles=[H3Title(title='Increased Visibility', content=''), H3Title(title='Higher Conversion Rates', content=''), H3Title(title='Competitive Advantage', content='')]), H2Title(title='Key Elements of SEO for E-commerce', content='', h3_titles=[H3Title(title='Keyword Research', content=''), H3Title(title='On-Page Optimization', content=''), H3Title(title='Link Building', content=''), H3Title(title='Mobile Optimization', content='')]), H2Title(title='Best Practices for Writing SEO Articles in E-commerce', content='', h3_titles=[H3Title(title='Unique and High-Quality Content', content=''), H3Title(title='Optimized Product Descriptions', content=''), H3Title(title='Utilizing Long-Tail Keywords', content=''), H3Title(title='Image Optimization', content='')]), H2Title(title='Conclusion', content='', h3_titles=None)])

In [47]:
class ParagraphWriter(BaseModel):
    """Component responsible for writing a paragraph based on provided information."""

    paragraph:str = Field(description="The written content of the H1 paragraph")

    @classmethod
    def get_system_prompt(cls) -> str:
        return """You are an expert SEO article writer. You will write the content of a paragraph based on the provided overall outline and paragraph title"""
    

result = class_tester(ParagraphWriter, "Overall article outline: {outline}\n\nParagraph title:\n\n {p_title}", {"outline": outline, "p_title": "Keyword Research for E-commerce SEO"})
result

ParagraphWriter(paragraph='Keyword research is a crucial aspect of e-commerce SEO. By identifying relevant keywords, online businesses can attract targeted traffic to their websites. Understanding the difference between long-tail and short-tail keywords is essential for optimizing content and improving search engine rankings. Effective keyword research lays the foundation for a successful SEO strategy in the competitive e-commerce landscape.')

In [56]:
for h2 in outline.h2_titles:
    h2.content = class_tester(ParagraphWriter, "Overall article outline: {outline}\n\nParagraph title:\n\n  {p_title}", {"outline": outline, "p_title": h2.title})


In [57]:
outline

OutlineGenerator(h2_titles=[H2Title(title='Introduction', content=ParagraphWriter(paragraph="Welcome to the world of E-commerce where the digital landscape is constantly evolving. In this fast-paced environment, the role of Search Engine Optimization (SEO) is more crucial than ever. SEO plays a vital role in ensuring that your online store is visible to potential customers and stands out amidst the competition. Let's delve into the significance of SEO in E-commerce and explore how it can drive success for your online business."), h3_titles=None), H2Title(title='Importance of SEO in E-commerce', content=ParagraphWriter(paragraph='Search Engine Optimization (SEO) is a critical component in the realm of E-commerce, playing a pivotal role in enhancing the visibility of online stores, boosting conversion rates, and providing a competitive edge in the digital marketplace. In this section, we will explore the significance of SEO in E-commerce and how it contributes to the success of online bu

In [58]:
def stitch_h2_paragraphs(outline: OutlineGenerator) -> str:
    article = ""

    for h2 in outline.h2_titles:
        # Add H2 title
        article += f"## {h2.title}\n\n"
        # Add H2 content
        article += f"{h2.content.paragraph}\n\n"
        # If there are H3 titles, add them
        if h2.h3_titles:
            for h3 in h2.h3_titles:
                # Add H3 title
                article += f"### {h3.title}\n\n"
                # Add H3 content
                article += f"{h3.content}\n\n"

    return article.strip()

In [60]:
article = stitch_h2_paragraphs(outline)
article

"## Introduction\n\nWelcome to the world of E-commerce where the digital landscape is constantly evolving. In this fast-paced environment, the role of Search Engine Optimization (SEO) is more crucial than ever. SEO plays a vital role in ensuring that your online store is visible to potential customers and stands out amidst the competition. Let's delve into the significance of SEO in E-commerce and explore how it can drive success for your online business.\n\n## Importance of SEO in E-commerce\n\nSearch Engine Optimization (SEO) is a critical component in the realm of E-commerce, playing a pivotal role in enhancing the visibility of online stores, boosting conversion rates, and providing a competitive edge in the digital marketplace. In this section, we will explore the significance of SEO in E-commerce and how it contributes to the success of online businesses.\n\n### Increased Visibility\n\n\n\n### Higher Conversion Rates\n\n\n\n### Competitive Advantage\n\n\n\n## Key Elements of SEO 